In [ ]:
%load 'ex3.m'

In [ ]:
%% Machine Learning Online Class - Exercise 3 | Part 1: One-vs-all

%  Instructions
%  ------------
% 
%  This file contains code that helps you get started on the
%  linear exercise. You will need to complete the following functions 
%  in this exericse:
%
%     lrCostFunction.m (logistic regression cost function)
%     oneVsAll.m
%     predictOneVsAll.m
%     predict.m
%
%  For this exercise, you will not need to change any code in this file,
%  or any other files other than those mentioned above.
%

%% Initialization
clear ; close all; clc

%% Setup the parameters you will use for this part of the exercise
input_layer_size  = 400;  % 20x20 Input Images of Digits
num_labels = 10;          % 10 labels, from 1 to 10   
                          % (note that we have mapped "0" to label 10)

%% =========== Part 1: Loading and Visualizing Data =============
%  We start the exercise by first loading and visualizing the dataset. 
%  You will be working with a dataset that contains handwritten digits.
%

% Load Training Data
fprintf('Loading and Visualizing Data ...\n')

load('ex3data1.mat'); % training data stored in arrays X, y
m = size(X, 1);

% Randomly select 100 data points to display
rand_indices = randperm(m);
sel = X(rand_indices(1:100), :);

displayData(sel);

fprintf('Program paused. Press enter to continue.\n');
pause;

%% ============ Part 2: Vectorize Logistic Regression ============
%  In this part of the exercise, you will reuse your logistic regression
%  code from the last exercise. You task here is to make sure that your
%  regularized logistic regression implementation is vectorized. After
%  that, you will implement one-vs-all classification for the handwritten
%  digit dataset.
%

fprintf('\nTraining One-vs-All Logistic Regression...\n')

lambda = 0.1;
[all_theta] = oneVsAll(X, y, num_labels, lambda);

fprintf('Program paused. Press enter to continue.\n');
pause;


%% ================ Part 3: Predict for One-Vs-All ================
%  After ...
pred = predictOneVsAll(all_theta, X);

fprintf('\nTraining Set Accuracy: %f\n', mean(double(pred == y)) * 100);



In [ ]:

clear ; close all; clc

%%% Setup the parameters you will use for this part of the exercise
input_layer_size  = 400;  % 20x20 Input Images of Digits
num_labels = 10;          % 10 labels, from 1 to 10   
                          % (note that we have mapped "0" to label 10)

%%% =========== Part 1: Loading and Visualizing Data =============
%  We start the exercise by first loading and visualizing the dataset. 
%  You will be working with a dataset that contains handwritten digits.
%

% Load Training Data
fprintf('Loading and Visualizing Data ...\n')

load('ex3data1.mat'); % training data stored in arrays X, y
m = size(X, 1);

% Randomly select 100 data points to display
rand_indices = randperm(m);
sel = X(rand_indices(1:100), :);

displayData(sel);



In [ ]:
clear ; close all; clc
input_layer_size  = 400;  % 20x20 Input Images of Digits
num_labels = 10;          % 10 labels, from 1 to 10   

fprintf('Loading and Visualizing Data ...\n')

load('ex3data1.mat'); % training data stored in arrays X, y



In [ ]:
imagesc(reshape(X(1,:)/max1,20,20), [-1,1])

In [ ]:
function showNum(X, bg_rows= 8)
% showNum displays a grid of random images of numbers
% ex is the starting number
    ex = 1
    m = size(X, 1); % get number of examples (rows)
    rand_indices = randperm(m);
    X = X(rand_indices(1:(bg_rows^2)), :);
    
    img_wh = 20;
    
    bg = ones(bg_rows * img_wh,img_wh);
    
    for i = ex:(ex+bg_rows-1) % show 3 examples 
        for j = ex:(ex + bg_rows -1)
            colormap(gray)
            r = X(j + bg_rows* (i-ex),:);
            r = r / max(abs(r)); % all values between -1 and 1
            offsetx = (i-ex)*img_wh;
            offsety = (j-ex)*img_wh;
            bg( (1:img_wh) + offsetx , (1:img_wh) + offsety) =  flipud(reshape(r,img_wh,img_wh));
        end
    end
    imagesc(bg, [-1,1]);
end

In [ ]:
showNum(X,12)

In [ ]:
%%file displayData.m
function [h, display_array] = displayData(X, example_width)
%%%DISPLAYDATA Display 2D data in a nice grid
%%%   [h, display_array] = DISPLAYDATA(X, example_width) displays 2D data
%%%   stored in X in a nice grid. It returns the figure handle h and the 
%%%   displayed array if requested.


% Set example_width automatically if not passed in
if ~exist('example_width', 'var') || isempty(example_width) 
	example_width = round(sqrt(size(X, 2)));
end

% Gray Image
colormap(gray);

% Compute rows, cols
[m n] = size(X);
example_height = (n / example_width);

% Compute number of items to display
display_rows = floor(sqrt(m));
display_cols = ceil(m / display_rows);

% Between images padding
pad = 1;

% Setup blank display
display_array = - ones(pad + display_rows * (example_height + pad), ...
                       pad + display_cols * (example_width + pad));

% Copy each example into a patch on the display array
curr_ex = 1;
for j = 1:display_rows
	for i = 1:display_cols
		if curr_ex > m, 
			%break; 
		end
		% Copy the patch
		
		% Get the max value of the patch
		max_val = max(abs(X(curr_ex, :)));
		display_array(pad + (j - 1) * (example_height + pad) + (1:example_height), ...
		              pad + (i - 1) * (example_width + pad) + (1:example_width)) = ...
						reshape(X(curr_ex, :), example_height, example_width) / max_val;
		curr_ex = curr_ex + 1;
	end
	if curr_ex > m, 
		%break; 
	end
end

% Display Image

h = imagesc(display_array, CLIMITS= [-1 1]);

% Do not show axis
axis image off

%%%%drawnow;

end


In [ ]:
% Gray Image
colormap(gray);
imagesc(1- ((2*eye(100)-1)),[-1,1])

In [ ]:
%load lrCostFunction.m

In [ ]:
%%file lrCostFunction.m
function [J, grad] = lrCostFunction(theta, X, y, lambda=0)
%LRCOSTFUNCTION Compute cost and gradient for logistic regression with 
%regularization
%   J = LRCOSTFUNCTION(theta, X, y, lambda) computes the cost of using
%   theta as the parameter for regularized logistic regression and the
%   gradient of the cost w.r.t. to the parameters. 

% Initialize some useful values
%% m = length(y); % number of training examples
[m,n] = size(X) ;
% You need to return the following variables correctly 
J = 0;
grad = zeros(size(theta));

% ====================== YOUR CODE HERE ======================
% Instructions: Compute the cost of a particular choice of theta.
%               You should set J to the cost.
%               Compute the partial derivatives and set grad to the partial
%               derivatives of the cost w.r.t. each parameter in theta
%
% Hint: The computation of the cost function and gradients can be
%       efficiently vectorized. For example, consider the computation
%
%           sigmoid(X * theta)
%
%       Each row of the resulting matrix will contain the value of the
%       prediction for that example. You can make use of this to vectorize
%       the cost function and gradient computations. 
%
% Hint: When computing the gradient of the regularized cost function, 
%       there're many possible vectorized solutions, but one solution
%       looks like:
%           grad = (unregularized gradient for logistic regression)
%           temp = theta; 
%           temp(1) = 0;   % because we don't add anything for j = 0  
%           grad = grad + YOUR_CODE_HERE (using the temp variable)
%




[m,n] = size(X);
J = sum(-   y  .*log(    sigmoid(X * theta)) 
        - (1-y).*log(1 - sigmoid(X * theta))
        ) / m + lambda / (2*m) * theta(2:3)'*theta(2:3);



g1 =  sum((sigmoid(X * theta) - y) .* X(:,1))/ m ;
g2na = sum((sigmoid(X * theta) - y) .* X(:,2:n))/ m ;
g2nb = lambda / m * theta(2:n)  ;

grad = [g1,g2na + g2nb'] ;

% =============================================================

grad = grad(:);

end


In [ ]:
%load oneVsAll.m

In [ ]:
%%file oneVsAll.m
function [all_theta] = oneVsAll(X, y, num_labels, lambda)
%ONEVSALL trains multiple logistic regression classifiers and returns all
%the classifiers in a matrix all_theta, where the i-th row of all_theta 
%corresponds to the classifier for label i
%   [all_theta] = ONEVSALL(X, y, num_labels, lambda) trains num_labels
%   logistic regression classifiers and returns each of these classifiers
%   in a matrix all_theta, where the i-th row of all_theta corresponds 
%   to the classifier for label i

% Some useful variables
m = size(X, 1);
n = size(X, 2);

% You need to return the following variables correctly 
all_theta = zeros(num_labels, n + 1);

% Add ones to the X data matrix
X = [ones(m, 1) X];

% ====================== YOUR CODE HERE ======================
% Instructions: You should complete the following code to train num_labels
%               logistic regression classifiers with regularization
%               parameter lambda. 
%
% Hint: theta(:) will return a column vector.
%
% Hint: You can use y == c to obtain a vector of 1's and 0's that tell use 
%       whether the ground truth is true/false for this class.
%
% Note: For this assignment, we recommend using fmincg to optimize the cost
%       function. It is okay to use a for-loop (for c = 1:num_labels) to
%       loop over the different classes.
%
%       fmincg works similarly to fminunc, but is more efficient when we
%       are dealing with large number of parameters.
%
% Example Code for fmincg:
%
%     % Set Initial theta
%     initial_theta = zeros(n + 1, 1);
%     
%     % Set options for fminunc
%     options = optimset('GradObj', 'on', 'MaxIter', 50);
% 
%     % Run fmincg to obtain the optimal theta
%     % This function will return theta and the cost 
%     [theta] = ...
%         fmincg (@(t)(lrCostFunction(t, X, (y == c), lambda)), ...
%                 initial_theta, options);
%
    initial_theta = zeros(n + 1, 1);
    options = optimset('GradObj', 'on', 'MaxIter', 50);
    for k = 1:num_labels
        all_theta(k,:) = fmincg (@(t)(lrCostFunction(t, X, (y == k), lambda)), initial_theta, options);       
    end    
    r = all_theta

% =========================================================================


end


In [ ]:
Xm = [ -1 -1 ; -1 -2 ; -2 -1 ; -2 -2 ; ...
          1 1 ;  1 2 ;  2 1 ; 2 2 ; ...
         -1 1 ;  -1 2 ;  -2 1 ; -2 2 ; ...
          1 -1 ; 1 -2 ;  -2 -1 ; -2 -2 ];
ym = [ 1 1 1 1 2 2 2 2 3 3 3 3 4 4 4 4 ]';
sprintf('%0.5f ', oneVsAll(Xm, ym, 4, 0.1));


In [ ]:
size(ym == 3)
X = [ones(20,1) (exp(1) * sin(1:1:20))' (exp(0.5) * cos(1:1:20))'];
size(sin(X(:,1) + X(:,2)) > 0)
t1 = sin(reshape(1:2:24, 4, 3));
t2 = cos(reshape(1:2:40, 4, 5));



In [ ]:
%load submit.m


Inputs:

$L$, a learner (training algorithm for binary classifiers)
samples $X$

labels $y$ where $y_i \in  \{1, … K\}$ is the label for the sample $X_i$

Output:

a list of classifiers $f_k$ for $k \in \{1, …, K\}$

Procedure:

For each $k$ in $\{1, …, K\}$:

Construct a new label vector $y_i = 1$ where $y_i = k, 0 (or −1)$ elsewhere

Apply $L$ to $X, y$ to obtain $f_k$

In [ ]:
%load predictOneVsAll.m

In [ ]:
%%file predictOneVsAll.m
function p = predictOneVsAll(all_theta, X)
%PREDICT Predict the label for a trained one-vs-all classifier. The labels 
%are in the range 1..K, where K = size(all_theta, 1). 
%  p = PREDICTONEVSALL(all_theta, X) will return a vector of predictions
%  for each example in the matrix X. Note that X contains the examples in
%  rows. all_theta is a matrix where the i-th row is a trained logistic
%  regression theta vector for the i-th class. You should set p to a vector
%  of values from 1..K (e.g., p = [1; 3; 1; 2] predicts classes 1, 3, 1, 2
%  for 4 examples) 

m = size(X, 1);
num_labels = size(all_theta, 1);

% You need to return the following variables correctly 
p = zeros(size(X, 1), 1);

% Add ones to the X data matrix
X = [ones(m, 1) X];

% ====================== YOUR CODE HERE ======================
% Instructions: Complete the following code to make predictions using
%               your learned logistic regression parameters (one-vs-all).
%               You should set p to a vector of predictions (from 1 to
%               num_labels).
%
% Hint: This code can be done all vectorized using the max function.
%       In particular, the max function can also return the index of the 
%       max element, for more information see 'help max'. If your examples 
%       are in rows, then, you can use max(A, [], 2) to obtain the max 
%       for each row.
%       

% i should be a vector containing the maximum for each sample in X
% all 
[v,p] = max(sigmoid(all_theta * X'))

p = p'


% =========================================================================


end


In [ ]:
[v,i] = max(reshape(1:24,6,4))

In [7]:
%%file predict.m
function p = predict(Theta1, Theta2, X)
%PREDICT Predict the label of an input given a trained neural network
%   p = PREDICT(Theta1, Theta2, X) outputs the predicted label of X given the
%   trained weights of a neural network (Theta1, Theta2)

% Useful values
m = size(X, 1);
num_labels = size(Theta2, 1);

% You need to return the following variables correctly 
p = zeros(size(X, 1), 1);

% ====================== YOUR CODE HERE ======================
% Instructions: Complete the following code to make predictions using
%               your learned neural network. You should set p to a 
%               vector containing labels between 1 to num_labels.
%
% Hint: The max function might come in useful. In particular, the max
%       function can also return the index of the max element, for more
%       information see 'help max'. If your examples are in rows, then, you
%       can use max(A, [], 2) to obtain the max for each row.
%


% Add ones to the X data matrix
X = [ones(m, 1) X];
[m,n] = size(X)

a1 = sigmoid(Theta1 * X') % 

a1p = [ones(m,1)'; a1]

[v,p] = max(sigmoid(Theta2 * a1p))
p = p';

% =========================================================================


end


Created file '/Users/jlatmann/Programming/octave/machine-learning-ex3/ex3/predict.m'.


# Quiz: Neural Networks - learning

In [16]:
reshape(([zeros(5,3)(:);ones(4,6)(:)](16:39)),4,6)

ans =

   1   1   1   1   1   1
   1   1   1   1   1   1
   1   1   1   1   1   1
   1   1   1   1   1   1



In [18]:
function cost = J(theta)
    cost = theta^3 
end

eps = 0.01
theta = 1
(J(theta + eps) - J(theta - eps)) / (2*eps)

eps =  0.010000
theta =  1
cost =  1.0303
cost =  0.97030
ans =  3.0001
